# Machine Learning Section
### Load and explore historical data

In [1]:
import pandas
from pandas import DataFrame, get_dummies
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn import model_selection

In [2]:
df_hist = pandas.read_excel("beer_sales_historical_data.xlsx")

In [3]:
df_hist[:5]

,Product,Sales,Cost Per Unit,Easter Included,Carnival Included,Christmas Included,Some Other Holiday?,4 Wk Avg Temp,4 Wk Avg Humidity,Sales M-1 weeks,Sales M-2 weeks,Sales M-3 weeks,Sales M-4 Weeks,Sales M-5 weeks
0,American Lager,51.9,1.6625,No,No,Yes,No,80.69,69.19,17.0,22.4,13.5,14.5,28.0
1,Pilsner,55.8,2.2725,No,No,Yes,No,80.69,69.19,2.4,2.2,2.0,1.4,0.5
2,Oktoberfest,3385.6,1.3475,No,No,Yes,No,80.69,69.19,301.8,188.8,101.4,81.6,213.8
3,Schwarzbier,63.5,1.6600,No,No,Yes,No,80.69,69.19,73.8,69.4,72.8,75.4,57.4
4,Amber Ale,181.1,1.8725,No,No,Yes,No,80.69,69.19,23.1,22.6,22.1,19.9,23.2


In [4]:
len(df_hist)

596

### Experiment (aka Cross Fold)

In [5]:
categorical_columns = ['Product','Easter Included','Carnival Included', 'Christmas Included',
                       'Some Other Holiday?']

In [6]:
df_hist = get_dummies(df_hist, prefix={k:"dmy_%s"%k for k in categorical_columns},
                      columns = list(categorical_columns))

In [7]:
experiments = {"Algorithm":["Ordinary Least Squares", "Regression Tree", "Random Forest", 
                            "Bagging"], 
               "Objects" : [LinearRegression(), DecisionTreeRegressor(), 
                            RandomForestRegressor(), BaggingRegressor()], 
               "Results":[]}

In [8]:
for o in experiments["Objects"]:
    experiments["Results"].append(
        model_selection.cross_val_score(o, y=df_hist["Sales"], 
                                        X=df_hist.drop("Sales", axis=1)).mean())

In [9]:
DataFrame(experiments).drop("Objects", axis=1).set_index("Algorithm")

,Results
Algorithm,
Ordinary Least Squares,0.446870
Regression Tree,0.755152
Random Forest,0.824913
Bagging,0.809650


### Fit and Predict

In [10]:
fitted = RandomForestRegressor().fit(y=df_hist["Sales"], X=df_hist.drop("Sales", axis=1))

In [11]:
df_carn_original = pandas.read_excel("carnivale_promotion_data.xlsx")
df_carn = get_dummies(df_carn_original, prefix={k:"dmy_%s"%k for k in categorical_columns},
                      columns = list(categorical_columns))
assert "Sales" not in df_carn.columns 
assert {"Sales"}.union(df_carn.columns).issubset(set(df_hist.columns))
len(df_carn)

36

We shouldn't be surprised that the much larger historical data table has categorical selections that don't appear in the Carnivale promotion table.

In [12]:
for fld in set(df_hist.columns).difference(df_carn.columns, {"Sales"}):
    assert fld.startswith("dmy_")
    df_carn[fld] = 0

In [13]:
predicted = fitted.predict(df_carn)

In [14]:
forecast_sales = df_carn_original[["Product", "Cost Per Unit"]].copy()
forecast_sales["Sales"] = predicted

# Optimization Section

In [15]:
from ticdat import TicDatFactory, Slicer
import gurobipy as gu

In [16]:
input_schema = TicDatFactory(products = [["Name"],["Family"]],
                             forecast_sales = [["Product", "Cost Per Unit"],
                                               ["Sales"]],
                             max_promotions = [["Product Family"],["Max Promotions"]]
                             )

In [17]:
dat = input_schema.TicDat(
        products = {'Amber Ale': 'Ale',
                    'American Lager': 'Lager',
                    'Bitter Ale': 'Ale',
                    'IPA': 'Ale',
                    'Oktoberfest': 'Lager',
                    'Pilsner': 'Lager',
                    'Porter': 'Ale',
                    'Schwarzbier': 'Lager',
                    'Stout': 'Ale'},
        forecast_sales = forecast_sales.set_index(["Product", "Cost Per Unit"]), 
        max_promotions = {'Ale': 2, 'Lager': 2})
assert (len(dat.forecast_sales) == len(forecast_sales))
assert (set(dat.products) == {pdct for (pdct, cost) in dat.forecast_sales})

In [18]:
normal_price = {pdct:0 for pdct in dat.products}
for pdct, price in dat.forecast_sales:
    normal_price[pdct] = max(normal_price[pdct], price)

It's natural that the predictions don't always imply that cheaper costs will result in more sales. This is more likely for this data set since the training data table was fairly small.

Here is the number of total discount options across all products.

In [19]:
len(dat.forecast_sales) - len(dat.products)

27

Here is the number of meaningful discounts (i.e. discounts that are predicted to increase sales).

In [20]:
number_meaningful_discounts = 0
for (pdct, price), r in dat.forecast_sales.items():
    if (price < normal_price[pdct] and 
        r["Sales"] > dat.forecast_sales[pdct,normal_price[pdct]]["Sales"]):
        number_meaningful_discounts += 1
number_meaningful_discounts

18

In [21]:
def revenue(pdct, price):
    return dat.forecast_sales[pdct, price]["Sales"] * price
def investment(pdct, price):
    return max(0, dat.forecast_sales[pdct, price]["Sales"] * normal_price[pdct] -
                  revenue(pdct, normal_price[pdct]))

In [22]:
mdl = gu.Model("beer promotion")

In [23]:
pdct_price = {(pdct, price):mdl.addVar(vtype = gu.GRB.BINARY, 
                                       name = "pdct_%s_price_%s"%(pdct, price))
              for pdct, price in dat.forecast_sales}

In [24]:
pp_slice = Slicer(pdct_price)
for pdct in dat.products:
    mdl.addConstr(gu.quicksum(pdct_price[_pdct, _price] 
                              for _pdct, _price in pp_slice.slice(pdct, '*')) == 1,
                  name = "pick_one_price_%s"%pdct)

In [25]:
pf_slice = Slicer((r["Family"], pdct) for pdct, r in dat.products.items())
for pdct_family, r in dat.max_promotions.items():
    mdl.addConstr(gu.quicksum(pdct_price[__pdct, __price]
                              for _pdct_family, _pdct in pf_slice.slice(pdct_family, '*')
                              for __pdct, __price in pp_slice.slice(_pdct, '*')
                              if __price != normal_price[__pdct]) <= r["Max Promotions"],
                  name = "max_promotions_%s"%pdct_family)

In [26]:
max_investment = 500
total_qty = mdl.addVar(name="total_qty")
total_revenue = mdl.addVar(name="total_revenue")
total_investment = mdl.addVar(name="total_investment", ub = max_investment)

In [27]:
mdl.addConstr(total_qty == gu.quicksum(dat.forecast_sales[pdct, price]["Sales"] * var
                                      for (pdct, price), var in pdct_price.items()))
mdl.addConstr(total_revenue == gu.quicksum(revenue(pdct, price)*var 
                                           for (pdct, price),var in pdct_price.items()))
mdl.addConstr(total_investment == gu.quicksum(investment(pdct, price)*var
                                              for (pdct, price),var in pdct_price.items()))

<gurobi.Constr *Awaiting Model Update*>

In [28]:
mdl.setObjective(total_qty, sense=gu.GRB.MAXIMIZE)
mdl.optimize()

Optimize a model with 14 rows, 39 columns and 156 nonzeros
Variable types: 3 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6108.3
Presolve removed 7 rows and 25 columns
Presolve time: 0.00s
Presolved: 7 rows, 14 columns, 38 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: objective 6.194809e+03, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6194.80934    0    2 6108.30000 6194.80934  1.42%     -    0s
H    0     0                    6182.8600000 6194.80934  0.19%     -    0s
     0     0     cutoff    0      6182.86000 6182.86382  0.00%     -    0s

Cutting planes:
  Cover: 1

Explored 0 nodes (6 simplex iterations) in 0.02 second

In [29]:
mdl.status == gu.GRB.OPTIMAL

True

Display the KPIs and pricing choices. Observe that the total investment constraint is nearly tight (likely as tight as possible given integrality of variables) and the max promotions constraint is tight for each product family.

In [30]:
(total_qty.X,  total_revenue.X, total_investment.X)

(6182.859999999999, 11629.061774999998, 472.22207499999956)

In [31]:
price_selections = {"Product":[], "Price":[], "Is Discount":[], "Family":[]}
for (pdct, price), var in pdct_price.items():
    if abs(var.X -1) < 0.0001: # i.e. almost one
        price_selections["Product"].append(pdct)
        price_selections["Price"].append(price)
        price_selections["Is Discount"].append(price < normal_price[pdct])
        price_selections["Family"].append(dat.products[pdct]["Family"])

In [32]:
(DataFrame(price_selections).set_index("Product")[["Price", "Is Discount", "Family"]].
 sort_values("Family"))

,Price,Is Discount,Family
Product,,,
Bitter Ale,2.5650,False,Ale
Stout,2.8475,True,Ale
Amber Ale,1.7500,False,Ale
Porter,3.8425,False,Ale
IPA,1.9150,True,Ale
Oktoberfest,1.2825,True,Lager
American Lager,1.5125,True,Lager
Schwarzbier,1.4700,False,Lager
Pilsner,2.2275,False,Lager
